In [1]:
import os
from pathlib import Path

NOTEBOOK_PATH: Path = Path(os.path.abspath(''))

# print(f'当前运行在：{NOTEBOOK_PATH} 目录下。')

PATH_FOR_RESULT: Path = NOTEBOOK_PATH.joinpath('RESULT')

if not PATH_FOR_RESULT.exists():
    PATH_FOR_RESULT.mkdir()

# 将数据写入数据库

数据库是个很大的话题，没概念的朋友把数据库理解为图书馆（房子——用于储存的空间、书——被储存的数据、工作人员——帮你找数据的机制）。

在我学习编程的年代，放眼望去的数据库几乎都是关系型数据库，现在还比较著名的有：[SQLite](https://www.sqlite.org/index.html)、MySQL（[下载](https://www.mysql.com/cn/products/community/)）、PostgreSQL（[下载](https://www.postgresql.org/download/)）。但是现在流行 no-sql 数据库，Redis、MongoDB 什么，可惜我不会。

数据库编程还有一个注意点：ORM。就是说，程序规模打了之后，程序里到处都是 sql 语句，就很不方便。因此会再套一层来把数据库的东西转换映射成 Python 的对象。

我们选择 PeeWee 作为 ORM，他的功能没有 SqlAlchemy 强大，但是简单。

SQLite 也很简单，简单到几乎承包了手机 App 对数据库的全部需求。如果只是通过 Python 使用 SQLite，都不需要安装其它软件。

## 定义日线数据对象




In [2]:
from peewee import (
    SqliteDatabase,
    Model,
    AutoField,
    CharField,
    FixedCharField,
    DateField,
    FloatField,
    IntegerField,
)


# 定义并打开 SQLite 数据库。
db: SqliteDatabase = SqliteDatabase(PATH_FOR_RESULT.joinpath('database.sqlite'))


# 定义基本ORM对象，可以少打一些字。
class ModelBase(Model):
    """
    ORM 基对象。
    """
    class Meta:
        database = db
        legacy_table_names = False


# 定义期货数据（日线）。
class FuturesQuoteDaily(ModelBase):
    """
    期货交易数据（日线）。
    """
    
    # id 可以理解为一张表的行号。
    id = AutoField(primary_key=True)
    
    symbol = FixedCharField(verbose_name='代码', max_length=6)
    date = DateField(verbose_name='日期')
    open = FloatField(verbose_name='开盘价', null=True)
    high = FloatField(verbose_name='最高价', null=True)
    low = FloatField(verbose_name='最低价', null=True)
    close = FloatField(verbose_name='收盘价')
    volume = IntegerField(verbose_name='成交量')
    amount = FloatField(verbose_name='成交额')
    settlement = FloatField(verbose_name='结算价')
    open_interest = IntegerField(verbose_name='持仓量')
    
    def __str__(self) -> str:
        """
        为 print 提供数据。
        """
        return f'<FuturesQuoteDaily(' \
               f'symbol={self.symbol}, ' \
               f'date={self.date}, ' \
               f'open={self.open}, ' \
               f'high={self.high}, ' \
               f'low={self.low}, ' \
               f'close={self.close}, ' \
               f'volume={self.volume}, ' \
               f'amount={self.amount}, ' \
               f'settlement={self.settlement}, ' \
               f'open_interest={self.open_interest}' \
               f')>'


# 定义期权数据（日线）。
class OptionQuoteDaily(ModelBase):
    """
    期权交易数据（日线）。
    """
    
    id = AutoField(primary_key=True)
    
    symbol = FixedCharField(verbose_name='代码', max_length=6)
    date = DateField(verbose_name='日期')
    open = FloatField(verbose_name='开盘价', null=True)
    high = FloatField(verbose_name='最高价', null=True)
    low = FloatField(verbose_name='最低价', null=True)
    close = FloatField(verbose_name='收盘价')
    volume = IntegerField(verbose_name='成交量')
    amount = FloatField(verbose_name='成交额')
    settlement = FloatField(verbose_name='结算价')
    open_interest = IntegerField(verbose_name='持仓量')
    delta = FloatField(verbose_name='delta')
    
    def __str__(self) -> str:
        """
        为 print 提供数据。
        """
        return f'<OptionQuoteDaily(' \
               f'symbol={self.symbol}, ' \
               f'date={self.date}, ' \
               f'open={self.open}, ' \
               f'high={self.high}, ' \
               f'low={self.low}, ' \
               f'close={self.close}, ' \
               f'volume={self.volume}, ' \
               f'amount={self.amount}, ' \
               f'settlement={self.settlement}, ' \
               f'open_interest={self.open_interest}, ' \
               f'delta={self.delta}' \
               f')>'


# 测试，创建空的数据表。
FuturesQuoteDaily.create_table()
OptionQuoteDaily.create_table()

# 关闭数据库。
db.close()

True

可以看到在 collect_data/RESULT 目录下新生成了一个名为 <database.sqlite> 的文件。

## 写入数据库

让我们用 collect_data/RESULT/unzipped/20200820_1.csv 做例子添加一些数据看看。

In [3]:
# 在【读取中国金融期货交易所（中金所）的历史数据】章节程序的基础上进行修改。

from typing import Dict, List, Tuple, Any
import datetime as dt
import csv

def read_cffex_data(data_file: Path) -> Tuple[List[Dict[str, Any]]]:
    """
    读取中国金融期货交易所（中金所，CFFEX）的历史交易数据 (csv 文件)。
    :param data_file: Path，待读取的文件。
    :return: Tuple，共两项，每一项都是一个列表。
             前一个是期货数据，后一个是期权数据。
             每一个列表都是一个字典，字典的 key 是 str，为 key 的字典。
    """
    result_futures: List[Dict[str, Any]] = []
    result_option: List[Dict[str, Any]] = []
    
    # 从文件名中获得日期。
    filename: str = data_file.name[:8]
    date: dt.date = dt.date(
        year=int(filename[:4]),
        month=int(filename[4:6]),
        day=int(filename[6:8])
    )
    
    # 打开 <data_file> 读取数据。
    with open(data_file, mode='r', encoding='gbk') as csv_file:
        reader = csv.DictReader(csv_file)
        
        # 按行循环读取。
        for row in reader:
            # 忽略 <合约代码> 列为 <小计>、<合计> 的行。
            if row['合约代码'] == '小计' or row['合约代码'] == '合计':
                continue
            
            # 捕捉异常，打印出错的行，以便在出现我们未考虑的情况时，提供调试信息用于修改。
            try:
                # 合约代码，去除两端的空白（空格）
                symbol = row['合约代码'].strip()
                
                # 合约代码长度不超过6，期货
                if len(symbol) <= 6:
                    result_futures.append(
                        {
                            'exchange': 'CFFEX',
                            'date': date,
                            'symbol': symbol,
                            'open': float(row['今开盘']) if len(row['今开盘']) > 0 else 0.0,
                            'high': float(row['最高价']) if len(row['最高价']) > 0 else 0.0,
                            'low': float(row['最低价']) if len(row['最低价']) > 0 else 0.0,
                            'close': float(row['今收盘']) if len(row['今收盘']) > 0 else 0.0,
                            'settlement': float(row['今结算']),
                            'previous_settlement': float(row['前结算']),
                            'volume': int(row['成交量']) if len(row['成交量']) > 0 else 0,
                            'amount': float(row['成交金额']) if len(row['成交金额']) > 0 else 0.0,
                            'open_interest': int(float(row['持仓量'])),
                            'change_on_close': float(row['涨跌1']),
                            'change_on_settlement': float(row['涨跌2']),
                            'change_on_open_interest': int(float(row['持仓变化'])),
                        }
                    )
                # 合约代码长度超过6，期权
                else:
                    result_option.append(
                        {
                            'exchange': 'CFFEX',
                            'date': date,
                            'symbol': symbol,
                            'open': float(row['今开盘']) if len(row['今开盘']) > 0 else 0.0,
                            'high': float(row['最高价']) if len(row['最高价']) > 0 else 0.0,
                            'low': float(row['最低价']) if len(row['最低价']) > 0 else 0.0,
                            'close': float(row['今收盘']) if len(row['今收盘']) > 0 else 0.0,
                            'settlement': float(row['今结算']),
                            'previous_settlement': float(row['前结算']),
                            'volume': int(row['成交量']) if len(row['成交量']) > 0 else 0,
                            'amount': float(row['成交金额']) if len(row['成交金额']) > 0 else 0.0,
                            'open_interest': int(float(row['持仓量'])),
                            'change_on_close': float(row['涨跌1']),
                            'change_on_settlement': float(row['涨跌2']),
                            'change_on_open_interest': int(float(row['持仓变化'])),
                            'delta': float(row['Delta']),
                        }
                    )
            except ValueError as e:
                print(f'读取文件 {csv_file} 时发生错误。发生错误的行内容为：\n\t{row}')
    
    return result_futures, result_option


# 测试
# 用前面第4轮解压的文件，选了一个离现在比较近的日期，内容比较多一些。
test_data_file: Path = PATH_FOR_RESULT.joinpath('unzipped', '20200820_1.csv')

# 读取数据。
quote_futures, quote_option = read_cffex_data(test_data_file)

# 期货。
for item in quote_futures:
    FuturesQuoteDaily.insert(
        symbol=item['symbol'],
        date=item['date'],
        open=item['open'],
        high=item['high'],
        low=item['low'],
        close=item['close'],
        settlement=item['settlement'],
        volume=item['volume'],
        amount=item['amount'],
        open_interest=item['open_interest']
    ).execute()

# 期权。
for item in quote_option:
    OptionQuoteDaily.insert(
        symbol=item['symbol'],
        date=item['date'],
        open=item['open'],
        high=item['high'],
        low=item['low'],
        close=item['close'],
        settlement=item['settlement'],
        volume=item['volume'],
        amount=item['amount'],
        open_interest=item['open_interest'],
        delta=item['delta']
    ).execute()

# 关闭数据库。
db.close()

True

可以安装一个 [SQLite 查看工具](https://sqlitebrowser.org/)来打开 <database.sqlite> 文件查看数据库中的数据。最好使用【只读打开】，不影响我们的程序操作数据库。

![DB Browser for SQLite](attachment/screenshot_db-browser-for-sqlite.png)

上面的期货日线数据类中，只定义了最基本的数据，其它内容其实都可以通过读取后计算。如果把交易所提供的数据甚至我们自己计算的全部保存下来，好处是以后使用的时候不需要再算一次，这样用起来速度快；缺点是占用硬盘空间。这里就涉及一个取舍或者选择：计算速度或者存储空间。

或许可以这样考虑：如果使用个人计算机，CPU速度一般，但是硬盘现在很便宜了，我们可以选择保存全部数据；如果使用云主机，CPU速度一般还不错，不过储存空间还是比较贵的，可以选择只保存必需的数据。

## 其它数据库形式

如果我们不想使用 SQLite 呢？peewee 还支持 MySQL（需要第三方库 MySQLdb 或者 pymysql）、PostgreSQL（需要第三方库 psycopg2）和 CockroachDB（这个从来没听过，不会用）。

MySQL 和 PostgreSQL 都是主从架构的，也就是有一台主机运行着服务器端程序，众多的用户通过客户端程序来访问数据库。当然，就你一个用户的话服务器端程序和客户端程序都跑在一台电脑上也没有问题的。

```
# 建立一个 MySQL 数据库连接。

db = peewee.MySQLDatabase(
    '数据库名字',
    user='用户名',
    password='密码',
    host='主机地址',
    port='主机端口'
)


# 建立一个 PostgreSQL 数据库连接。

db = peewee.PostgresqlDatabase(
    '数据库名字',
    user='用户名',
    password='密码',
    host='主机地址',
    port='主机端口'
)
```

其中，<数据库名字>是在数据库设置好之后填在这里的。具体可以搜一下教程。

创建数据库连接的过程能不能被进一步的泛化？比如，我可能在家里用的是本地的 SQLite 数据库，在星巴克带着 MacBook Pro 装X的时候用的是阿里云的 PostgreSQL 数据库？

**As your wish.**

首先我们定义一个保存配置文件夹，保存在 C:\Users\<你的用户名>\ （Linux 和 macOS 是 /Home）下面，起个名字就做 <.investment>。前面加 <.> 是 Linux 的习惯，这样就隐藏了文件或者文件夹。

配置文件采用 JSON 格式，Python 自带处理模块。就命名为 <config.json>。

并且我们编写一个生成配置文件的程序。

In [4]:
import json


def load_json(json_file: Path) -> dict:
    """
    加载 json 文件为 dict。
    :param json_file: Path，json 文件的路径。
    :return: dict.
    """
    with open(json_file, mode='r', encoding='utf-8') as f:
        data = json.load(f)
    return data


def save_json(json_file: Path, data: dict) -> None:
    """
    将字典保存为 json 文件。
    :param json_file: Path，json 文件的路径。
    :param data: dict，要保存的数据。
    :return: None
    """
    with open(json_file, mode='w+', encoding='utf-8') as f:
        json.dump(data, f, indent=4, ensure_ascii=False)


def create_default_config() -> None:
    """
    创建默认的配置文件。
    :return: None
    """
    default_config: Dict[str, Any] = {
        'database': {
            'driver': 'SQLite',
            'database': 'investment.sqlite',
            # 以下项对 SQLite 无效，保留在这里将会写入配置文件，方便以后手工修改。
            'host': '127.0.0.1',
            'port': '5432',
            'user': 'user',
            'password': 'password',
        }
    }
    save_json(CONFIG_FILE, default_config)


# 定义配置文件夹路径。
CONFIG_PATH: Path = Path.home().joinpath(f'.investment')

# 如果配置文件夹不存在，创建它。
if not CONFIG_PATH.exists():
    CONFIG_PATH.mkdir()

# 定义配置文件路径。
CONFIG_FILE: Path = CONFIG_PATH.joinpath('config.json')

# 如果配置文件不存在，创建默认的版本。
if not CONFIG_FILE.exists():
    create_default_config()

# 读取配置文件。
CONFIGS: Dict[str, Any] = load_json(CONFIG_FILE)

# 打印出来看看
print(CONFIGS)

{'database': {'driver': 'SQLite', 'database': 'investment.sqlite', 'host': '127.0.0.1', 'port': '5432', 'user': 'user', 'password': 'password'}}


如果不使用 SQLite，默认配置文件应该是：

**MySQL**

```
'database': {
    'driver': 'MySQL',
    # 以下按照数据库配置填写
    'database': 'investment.sqlite',  # 数据库名称
    'host': '127.0.0.1',              # 主机地址
    'port': '5432',                   # 端口号
    'user': 'user',                   # 用户名
    'password': 'password',           # 密码
}
```

**PostgreSQL**

```
'database': {
    'driver': 'PostgreSQL',
    # 以下按照数据库配置填写
    'database': 'investment.sqlite',  # 数据库名称
    'host': '127.0.0.1',              # 主机地址
    'port': '5432',                   # 端口号
    'user': 'user',                   # 用户名
    'password': 'password',           # 密码
}
```

接下来编写建立数据库连接的程序：

In [5]:
from typing import Callable

from peewee import (
    Database,
    SqliteDatabase,
    MySQLDatabase,
    PostgresqlDatabase,
)


# 定义一个变量类型简写。
DATABASE_SETTINGS = Dict[str, str]


def create_sqlite_database(settings: DATABASE_SETTINGS) -> SqliteDatabase:
    """
    创建 SQLite 数据库连接。
    :param settings: DATABASE_SETTINGS 类型（简写的Dict[str, str]）。
    :return: peewee.SqliteDatabase 对象。
    """
    return SqliteDatabase(
        Path(settings['database']),
        pragmas={
            'journal_mode': 'wal',
            'encoding': 'utf8',
            'cache_size': -640000,  # 64MB
            'foreign_keys': 1,      # Enforce foreign-key constraints
        }
    )


def create_mysql_database(settings: DATABASE_SETTINGS) -> MySQLDatabase:
    """
    创建 PostgreSQL 数据库连接。
    :param settings: DATABASE_SETTINGS 类型（简写的Dict[str, str]）。
    :return: peewee.MySQLDatabase 对象。
    """
    return MySQLDatabase(
        settings['database'],
        user=settings['user'],
        password=settings['password'],
        host=settings['host'],
        port=settings['port']
    )


def create_postgresql_database(settings: DATABASE_SETTINGS) -> PostgresqlDatabase:
    """
    创建 PostgreSQL 数据库连接。
    :param settings: DATABASE_SETTINGS 类型（简写的Dict[str, str]）。
    :return: peewee.PostgresqlDatabase 对象。
    """
    return PostgresqlDatabase(
        settings['database'],
        user=settings['user'],
        password=settings['password'],
        host=settings['host'],
        port=settings['port']
    )


def create_database() -> Database:
    """
    从配置文件读取数据配置，返回数据库连接。
    :return: peewee.Database 对象。
    """
    driver_mapper: Dict[str, Callable] = {
        'SQLITE': create_sqlite_database,
        'MYSQL': create_mysql_database,
        'POSTGRESQL': create_postgresql_database,
    }

    driver: str = CONFIGS['database']['driver'].upper()

    assert driver in driver_mapper.keys()

    return driver_mapper[driver](CONFIGS['database'])


# db 就是数据库连接。
db: Database = create_database()

# 打印一下看看。
print(db)

# 用完就关闭数据库。
if not db.is_closed():
    db.close()

打印出来的似乎不是给人类准备的信息，不过开头确实是 peewee.SqliteDatabase，符合我们的配置。

下一步，我们完成一个[完整的流程](completed_flow.ipynb)。